In [1]:
!pip install pandas
import pandas as pd
!pip install numpy
import numpy as np

In [5]:
import os
import pandas as pd
os.getcwd()

path = "c:\\Users\\esthelle.kuissu\\OneDrive - Exiom Partners\\Documents\\Code_rapport_stage\\Bases sinistres"


In [13]:
import pandas as pd

# 📥 Chargement des deux bases
df_proba = pd.read_csv(path +  "\\base_predictions_sinistres_complete.csv", sep=";", encoding="utf-8")
df_bdd = pd.read_csv(path + "\\bdd_sinistres_MODELPROBA.csv", sep=";", encoding="utf-8")


# 🔗 Jointure sur 'cle_sin'
df_fusion = df_bdd.merge(
    df_proba[["cle_sin", "sinistre_id_reel", "proba_sinistre_0", "proba_sinistre_1"]],
    on="cle_sin",
    how="left"  # ← garde tous les individus de la bdd principale
)

# 💾 Sauvegarde de la base enrichie
df_fusion.to_csv(path + "\\base_sinistres_enrichie.csv", sep=";", encoding="utf8", index=False)
print("✅ Base fusionnée enregistrée sous : base_sinistres_enrichie.csv")

C:\Users\esthelle.kuissu\AppData\Local\Temp\ipykernel_23920\1501822706.py:5: DtypeWarning: Columns (24,83) have mixed types. Specify dtype option on import or set low_memory=False.
  df_bdd = pd.read_csv(path + "\\bdd_sinistres_MODELPROBA.csv", sep=";", encoding="utf-8")


✅ Base fusionnée enregistrée sous : base_sinistres_enrichie.csv


In [16]:
# 📦 Liste des variables à supprimer
vars_to_drop = [
    "date_evt", "date_evt_decla", "date_premsous", "date_dernrempl", "date_nais", "age", "date_permis", "anc_permis",
    "profess", "sitfam", "orig_attest", "aac", "date_sortie_veh", "age_veh", "usage2", "franch", "rachat_francBG",
    "garag", "marq", "modele2", "generation", "modele_desc", "caros", "segment", "energ", "num_sra", "pdyne", "pfisc",
    "vites", "pvid", "poidspuiss", "sport", "frein", "trans", "alim", "suspe", "boite", "couple", "airbag", "crash",
    "co2min", "elt_secu", "vu", "bureau_ratt", "fract_paiem_soc", "formule_gar", "systeme", "exercice", "GARANTIE_HOST",
    "MT_FRANCHISE", "MT_CHARGE_BRUTE", "MT_CHARGE_NETTE", "MT_REGLEMENT_SIN", "MT_REGLEMENT_COM", "MT_ABANDON_RECOURS",
    "MT_FRAIS_HONO", "MT_PROVISIONS", "MT_RECOURS_TOTAL", "MT_ESTIMATION_RECOURS_RESTANTS", "MT_AVANCE_RECOURS",
    "MT_AVANCE_RECOURS_RECOURUS", "MT_PAIEMENTS", "MT_CHARGE_BRUTE_ECO", "T_CHARGE_NETTE_ECO", "ZN_NO_RAPPORT_EXPTIS",
    "MT_VAL_REMPL_VEH_TTC", "NB_REP", "MT_REMISE", "MT_FORFAIT", "MT_FORFAIT_ERD", "LIB_CHOC", "LIB_INTENS",
    "GARAGE_AGREE_O_N", "DPT_REP", "RESEAU", "NB_PIECE_REMPLACEE", "NB_PIECE_REPAREE", "veh_info_manquante",
    "date_ord", "date_sortie_veh_imputee", "annee_sortie_veh"
]

# 🧹 Suppression des colonnes
df_cleaned = df_fusion.drop(columns=vars_to_drop, errors="ignore")

# 💾 Sauvegarde de la version nettoyée
df_cleaned.to_csv(path + "base_sinistres_nettoyee.csv", index=False)
print("✅ Base nettoyée enregistrée sous : base_sinistres_nettoyee.csv")

✅ Base nettoyée enregistrée sous : base_sinistres_nettoyee.csv


In [17]:
# 🔍 Liste des variables explicatives (exclure la cible et les colonnes techniques)
exclude_cols = ["MT_TOTAL_REPARABLE", "MT_MO_REPARABLE", "MT_IP_REPARABLE",  "MT_PIECES_REPARABLE", "sinistre_id_reel", "cle_sin", "proba_sinistre_0", "proba_sinistre_1"]
variables = [col for col in df_cleaned.columns if col not in exclude_cols]

# 🧠 Construction de la formule GLM
formula_glm = "MT_TOTAL_REPARABLE ~ " + " + ".join(variables)
print("📋 Formule GLM utilisée :")
print(formula_glm)

📋 Formule GLM utilisée :
MT_TOTAL_REPARABLE ~ clos + sans_suite + quart_resp + formule_ratt + cond_indet + sexe + cylin + pkw + cdval + val + pickup + luxe + prestige + soumis_crm + crm + boncond + nab50 + petit_rouleur + MT_CHARGE_NETTE_ECO + NBHH_MO_REPARABLE + TAUX_HORAIRE_MO + NBHH_IP_REPARABLE + NB_PIECES_REEMPLOI + MT_PIECES_REEMPLOI + RE_AVEC_PIECE_REEMPLOI + RE_AVEC_PIECE_REPAR_MP + RE_AVEC_PIECE_REMPL_MP + _RESEAU_ + date_nais_f + date_permis_f + formule_ratt_g + sitfam_g + usage2_g + orig_attest_g + caros_g + segment_g + energ_g + frein_g + alim_g + suspe_g + crash_g + fract_paiem_soc_g + cdval_g + modele2_g + profess_g + marq_g + garag_g + Region + Zone


In [21]:
! pip install statsmodels

  Using cached patsy-1.0.1-py2.py3-none-any.whl.metadata (3.3 kB)
   ---------------------------------------- 0.0/9.6 MB ? eta -:--:--
   ------------- -------------------------- 3.1/9.6 MB 16.8 MB/s eta 0:00:01
   ------------------------------- -------- 7.6/9.6 MB 19.6 MB/s eta 0:00:01
   ---------------------------------------- 9.6/9.6 MB 18.2 MB/s eta 0:00:00
Using cached patsy-1.0.1-py2.py3-none-any.whl (232 kB)

   ---------------------------------------- 0/2 [patsy]
   ---------------------------------------- 0/2 [patsy]
   ---------------------------------------- 0/2 [patsy]
   -------------------- ------------------- 1/2 [statsmodels]
   -------------------- ------------------- 1/2 [statsmodels]
   -------------------- ------------------- 1/2 [statsmodels]
   -------------------- ------------------- 1/2 [statsmodels]
   -------------------- ------------------- 1/2 [statsmodels]
   -------------------- ------------------- 1/2 [statsmodels]
   -------------------- --------------

In [23]:
import statsmodels.api as sm

In [22]:
import statsmodels.formula.api as smf

In [26]:
# 📊 GLM pour sinistres de type 1
df_type_1 = df_cleaned[df_cleaned["sinistre_id_reel"] == 1]
glm_1 = smf.glm(
    formula=formula_glm,
    data=df_type_1,
    family=sm.families.Gamma(sm.families.links.log())
).fit()

# 📊 GLM pour sinistres de type 0
df_type_0 = df_cleaned[df_cleaned["sinistre_id_reel"] == 0]
glm_0 = smf.glm(
    formula=formula_glm,
    data=df_type_0,
    family=sm.families.Gamma(sm.families.links.log())
).fit()


c:\Users\esthelle.kuissu\OneDrive - Exiom Partners\Documents\Code_rapport_stage\EX_RAP_STAGE\Exiom_Stage\.venv\Lib\site-packages\statsmodels\genmod\families\links.py:13: FutureWarning: The log link alias is deprecated. Use Log instead. The log link alias will be removed after the 0.15.0 release.
  warnings.warn(


MemoryError: Unable to allocate 6.29 GiB for an array with shape (29066, 29065) and data type float64

# STOP ICI

In [58]:
Y1 = dataM['MT_MO_REPARABLE']

In [59]:
X1 = dataM[['pkw', 'age_veh', 'val', 'generation', 'airbag', 'co2min', 'anc_permis',
       'cylin', 'poidspuiss', 'crm', 'nab50','NBHH_MO_REPARABLE', 'TAUX_HORAIRE_MO']]

In [60]:
print(X1.dtypes)


pkw                  float64
age_veh              float64
val                  float64
generation           float64
airbag               float64
co2min               float64
anc_permis           float64
cylin                float64
poidspuiss           float64
crm                  float64
nab50                float64
NBHH_MO_REPARABLE    float64
TAUX_HORAIRE_MO      float64
dtype: object


In [61]:
#!pip install pyglmnet
!pip install scipy
from pyglmnet import GLM
from scipy import stats

In [62]:
from pyglmnet import GLM
model = GLM(distr='gamma', alpha=1, reg_lambda=0.01)
import numpy as np

In [63]:
X1 = X1.to_numpy()  # Convertir DataFrame en ndarray
Y1 = Y1.to_numpy()  # Convertir Series en ndarray


In [64]:
model.fit(X1, Y1)  # Réessayer l'ajustement du modèle


c:\Users\esthelle.kuissu\OneDrive - Exiom Partners\Documents\Code_rapport_stage\Note_book\.venv\Lib\site-packages\pyglmnet\pyglmnet.py:92: RuntimeWarning: overflow encountered in exp
  mu = np.log1p(np.exp(z))
c:\Users\esthelle.kuissu\OneDrive - Exiom Partners\Documents\Code_rapport_stage\Note_book\.venv\Lib\site-packages\pyglmnet\pyglmnet.py:863: UserWarning: Reached max number of iterations without convergence.
  warnings.warn(


<
Distribution | gamma
alpha | 1.00
max_iter | 1000.00
lambda: 0.01
>

In [65]:
# Predicting the response variable using the fitted model
Y_pred = model.predict(X1)

In [66]:
dataM['Y_pred'] = Y_pred  # Ajouter les prédictions au DataFrame original
# Calculer les résidus 
dataM['residuals'] = dataM['MT_MO_REPARABLE'] - dataM['Y_pred']
# Afficher les 5 premières lignes du DataFrame avec les prédictions et les résidus
print(dataM[['MT_MO_REPARABLE', 'Y_pred', 'residuals']].head(10))


   MT_MO_REPARABLE      Y_pred   residuals
0          598.300         inf        -inf
1           65.000         inf        -inf
2           70.000         inf        -inf
3          227.500  517.094524 -289.594524
4          550.025  677.437458 -127.412458
5          488.025  572.001228  -83.976228
6         1237.000         inf        -inf
7          108.340         inf        -inf
8           62.400         inf        -inf
9          133.000         inf        -inf


In [ ]:
!pip install pyglmnet



In [13]:
!pip install distutils

ERROR: Could not find a version that satisfies the requirement distutils (from versions: none)
ERROR: No matching distribution found for distutils


In [18]:
pip install setuptools

  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
Using cached setuptools-80.9.0-py3-none-any.whl (1.2 MB)
Note: you may need to restart the kernel to use updated packages.


In [20]:
pip install --upgrade setuptools


Note: you may need to restart the kernel to use updated packages.


In [21]:
from setuptools import setup
